<a href="https://colab.research.google.com/github/aqeel-12/Bantu-Language-Modeling/blob/main/nlp_04_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Approach of the Code**

======================================================================= 1 Loading   
1) Load train dataset  

======================================================================= 2 Cleaning  
2) Prepare the useful characters: ASCII_Chars  
3) Clean dataset:  
  - Remove punctuation characters  
  - List of all the characters without losing the order of the characters  

======================================================================= 3 Training     
4) Find the ngrams & ngram_nextCharacters of these characters  
5) Find ngrams count & ngram_nextCharacters count and store them  

======================================================================= 4 Probability  
6) Find the probability of ngram & nextChar  

======================================================================= 5 Evaluating  
8) For each given history & nextChar,   
  - get the probability of history(ngram) and nextChar  
  - apply -log2 on probability  
  - summation of all these gives result as log2loss.  

### **Defining Language Model**

In [ ]:
import re
import os
import sys
import numpy as np
import time
import math
from datetime import datetime

In [ ]:
L = list("ngrams_nextChars are stored successfully")
n=3
ngram = 3*["<START>"] # ('<START>', '<START>', '<START>')
for char in L:  
  totalLoss += -log2(ngrams_nextChars[('<START>', '<START>', '<START>'), 'n')]/ngrams[('<START>', '<START>', '<START>')])
  ngram = ngram[1:]+[char]
return totalLoss


['<START>', '<START>', '<START>', 'n', 'g', 'r', 'a', 'm', 's', '_', 'n', 'e', 'x', 't', 'C', 'h', 'a', 'r', 's', ' ', 'a', 'r', 'e', ' ', 's', 't', 'o', 'r', 'e', 'd', ' ', 's', 'u', 'c', 'c', 'e', 's', 's', 'f', 'u', 'l', 'l', 'y']


In [ ]:
# ngram, char
#ngram = tuple(ngram[1:]+[char])
#char = "g"
ngram, char
totalLoss += -log2(prob((('<START>', '<START>', 'n'), 'g'))/ prob(('<START>', '<START>', 'n')))

(('<START>', '<START>', 'n'), 'g')

In [ ]:
class DictionaryFunctions:
  def __init__(self) -> None:
    pass
  def addVal_to_dictKey(self, D, k, val=1): # Inputs: Dictionary, Key, Value; Ouput: Dictionary (Updated)
    if k in D:
      if type(val)==int:
        D[k] += val
      elif (type(val)==str):
        D[k].append(val)
    else:
      if type(val)==int:
        D[k] = val
      elif (type(val)==str):
        D[k] = [val]
    return D

  def find_probs_of_dict(self, D):
    total = 0
    for key in D.keys():
      total += D[key]
    for key in D.keys():
      D[key] /= total
    return D 

  def get_dict_from_list(self, L):
    D = {}
    for char in L:
      if char in D:
        D[char] += 1
      else:
        D[char] = 1
    return D

  def max_of_dict(self, D):  
    maxProb = 0
    maxKey = ''
    for key, probVal in D.items():
      if probVal>maxProb:
        maxProb = probVal
        maxKey = key
    return (maxKey, maxProb)

In [ ]:
class Language_Model:
  def __init__(self, n, lang="") -> None:
    self.n = n
    self.lang = lang
    self.DF = DictionaryFunctions()
    self.ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
    self.sentence_EndChars = '.!?'
    self.ngrams = {}
    self.ngrams_nextChars = {}
    self._ngrams_List = []
    self._ngrams_nextChars_List = []
    self._ngrams_possChars = {}
    self.startPad = ['<START>']
    self.endPad = ['<END>']
    self._ngrams_Available = []
    self._ngrams_nextChars_Available = []
    self._ngrams_notAvailable = []
    self._ngrams_nextChars_notAvailable = []
    self.train_chars_len = None
    self.train_char_prob = None
  
  def _char_prob(self, chars_List):
    print(chars_List)
    char_prob = self.DF.get_dict_from_list(chars_List)
    char_prob = self.DF.find_probs_of_dict(char_prob)
    return char_prob

  def _get_ngrams(self, text):
    n = self.n
    chars_tokens = (n)*self.startPad + text + (n)*self.endPad
    ngrams_nextChars = [(tuple(chars_tokens[i:i+n]),chars_tokens[i+n]) for i in range(len(chars_tokens)-n)]
    return ngrams_nextChars

  def clean_data(self, dataset: str) -> list:  
    ASCII_Chars = self.ASCII_Chars
    all_characters_in_data = re.findall(r"[%s]"%ASCII_Chars, dataset)
    # data_clean = "".join(all_characters_in_data)
    self.train_chars_len = len(all_characters_in_data)
    return all_characters_in_data
    
  def get_Sentences_from_Text(self, text):
    if type(text)==list:
      all_characters_in_data = text
    elif type(text) == str:
      all_characters_in_data = list(text)
    else:
      return "Please pass input either 'List_of_Characters' or 'Text_String'."
    endChars = self.sentence_EndChars # endChars=".!?" ; where '.' -> (period), '!' -> (exclamation mark) and '?' -> (question mark)
    sentences = []
    sentence = []
    for c in all_characters_in_data:
      if c not in endChars:
        sentence.append(c)
      else:
        sentence.append(c)
        sentences.append(sentence)
        sentence = []
    return sentences
  
  def store_ngrams(self, sentences):
    for sent in sentences:
      self.train_char_prob = self._char_prob(sent)
      ngrams_nextChars = self._get_ngrams(sent)

      for ngram_nextChar in ngrams_nextChars: # ((ngram, nextchar), )
        self._ngrams_nextChars_List.append(ngram_nextChar)
        self._ngrams_List.append(ngram_nextChar[0])
        self._ngrams_possChars = self.DF.addVal_to_dictKey(self._ngrams_possChars, ngram_nextChar[0], str(ngram_nextChar[1]))

    self.ngrams_nextChars = self.DF.get_dict_from_list(self._ngrams_nextChars_List)
    self.ngrams = self.DF.get_dict_from_list(self._ngrams_List)
    
    # conProb((ngram1, 'c')) = ngrams_nextChars[(ngram1, 'c')]/ngrams[ngram1]
    # "ngrams_nextChars are stored successfully"

    return "Success: ngrams & ngrams_nextChars are stored successfully"
  
  def fit(self, data):
    all_characters_in_data = self.clean_data(data)
    # sentences = self.get_Sentences_from_Text(all_characters_in_data)
    statusMessage = self.store_ngrams([all_characters_in_data])
    print("==== TRAINING IS COMPLETED ====")
    print(statusMessage)
  
  def get_prob(self, ngram, char):
    if ngram in self.ngrams:
      A = self.ngrams[ngram]
      self._ngrams_Available.append(ngram)
    else:
      A = 0
      self._ngrams_notAvailable.append(ngram)
    
    ngram_nextChar = tuple((ngram, char))
    if ngram_nextChar in self.ngrams_nextChars:
      B = self.ngrams_nextChars[ngram_nextChar]
      self._ngrams_nextChars_Available.append(ngram_nextChar)
    else:
      B = 0
      self._ngrams_nextChars_notAvailable.append(ngram_nextChar)
    
    #if A!=0:
    #  result = float(B/A)
    #else:
    #  result = 0
    result = float((B+1)/(A+len(ngram))) # self.train_chars_len
    return result
  
  def evaluate(self, text: str):
    text = self.clean_data(text)
    self._ngrams_Available = []
    self._ngrams_nextChars_Available = []
    self._ngrams_notAvailable = []
    self._ngrams_nextChars_notAvailable = []
    
    total_log2loss = 0
    ngram = self.n * self.startPad
    inputList = list(text)
    for char in text:
      result = self.get_prob(tuple(ngram), char)
      if result != 0:
        total_log2loss -= np.log2(result)
      
      ngram = ngram[1:]+[char]
    return total_log2loss/len(inputList)
  
  def evaluation_Status(self):
    print(f"Not available 'ngrams' are : {'-'*10}")
    print(self._ngrams_notAvailable)
    print(f"Not available 'ngrams_nextChars' are : {'-'*10}")
    print(self._ngrams_nextChars_notAvailable)


### **Load the Datasets**

In [ ]:
def load_data(filePath):
  data_cwe = open(filePath, 'r').read().lower()
  return data_cwe

In [ ]:
filePath = "/content/drive/MyDrive/NLP/Assignment03/data/cwe-train.txt"
train_data = load_data(filePath)

In [ ]:
filePath = "/content/drive/MyDrive/NLP/Assignment03/data/cwe-test.txt"
test_data = load_data(filePath)

In [ ]:
n=15
LM_15 = Language_Model(n)
LM_15.fit(train_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully


In [ ]:
print(f"Training Log2Loss : {LM_15.evaluate(train_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Training Log2Loss : 2.7864590801922526
_ngrams_Available :  603431
_ngrams_nextChars_Available :  603431
_ngrams_notAvailable :  0
_ngrams_nextChars_notAvailable :  0


In [ ]:
print(f"Testing Log2Loss : {LM_15.evaluate(test_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Testing Log2Loss : 3.5794126674101725
_ngrams_Available :  15920
_ngrams_nextChars_Available :  13311
_ngrams_notAvailable :  45796
_ngrams_nextChars_notAvailable :  48405


**Rough**

In [ ]:
train_data_Char_List = list(train_data)

DF = DictionaryFunctions()
train_data_Char_Dict = DF.get_dict_from_list(train_data_Char_List)
train_data_Char_Dict

{'c': 5446,
 'h': 20755,
 'i': 46266,
 'k': 23425,
 'a': 87267,
 'l': 33255,
 'e': 32796,
 ' ': 84914,
 'v': 5778,
 'n': 38966,
 'o': 26873,
 'g': 16766,
 'f': 2446,
 'y': 17481,
 'w': 31352,
 'u': 40148,
 'b': 7463,
 "'": 2502,
 'm': 25905,
 's': 10737,
 ',': 7106,
 'j': 1545,
 'd': 9800,
 't': 7034,
 '.': 5269,
 'p': 1141,
 'z': 8054,
 '"': 2167,
 '!': 341,
 '?': 421,
 '-': 12,
 '\n': 1}

In [ ]:
DF = DictionaryFunctions()
train_data_Char_Prob = DF.find_probs_of_dict(train_data_Char_Dict)
train_data_Char_Prob

{'c': 0.009025043418313911,
 'h': 0.03439492768033515,
 'i': 0.07667143936682178,
 'k': 0.0388196184491376,
 'a': 0.14461778626257804,
 'l': 0.05510977210356759,
 'e': 0.0543491230163465,
 ' ': 0.14071842394834877,
 'v': 0.009575229686194965,
 'n': 0.06457397022365403,
 'o': 0.04453360113484204,
 'g': 0.027784406527993213,
 'f': 0.004053480756738124,
 'y': 0.028969295629002106,
 'w': 0.051956144188574686,
 'u': 0.0665327659123149,
 'b': 0.012367590714446697,
 "'": 0.004146283259754206,
 'm': 0.042929443582706915,
 's': 0.01779322276577974,
 ',': 0.011775974757719179,
 'j': 0.00256035477071153,
 'd': 0.016240438027814236,
 't': 0.01165665725384136,
 '.': 0.00873172122128094,
 'p': 0.0018908509989526575,
 'z': 0.013346988558777128,
 '"': 0.0035911254292115766,
 '!': 0.0005651009558657811,
 '?': 0.0006976759601744687,
 '-': 1.9886250646303147e-05,
 '\n': 1.6571875538585954e-06}

In [ ]:
sum = 0
for key in train_data_Char_Prob:
  sum += train_data_Char_Prob[key]
print(sum)

1.0000000000000002


In [ ]:
test_data_Char_List = list(test_data)

DF = DictionaryFunctions()
test_data_Char_Dict = DF.get_dict_from_list(test_data_Char_List)
test_data_Char_Prob = DF.find_probs_of_dict(test_data_Char_Dict)

sum = 0
for key in test_data_Char_Prob:
  sum += test_data_Char_Prob[key]
print(sum)

0.9999999999999999
